# Project HIVE

Project Name: Fertilizer Recommendation System for Disease Prediction

Team ID: PNT2022TMID35945

Domain: AI

## Import/Upgrade libraries

In [1]:
!pip install tensorflow==2.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 4.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2


In [2]:
import tensorflow as tf

2022-11-21 17:18:13.897825: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/tensorflow


## Unzip HDF5 model

**NOTE: Model has been already trained on Kaggle with GPU and saved**

In [21]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='UABqLLIevO40IXgjH3qkbaGzQ2rHRWxCDk5mSGplg36j',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.ap.cloud-object-storage.appdomain.cloud')

bucket = 'projecthive-donotdelete-pr-ehbzvmp3k80cid'
object_key = 'hive.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [22]:
from io import BytesIO
import zipfile 
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')

file_paths=unzip.namelist() 
for path in file_paths: 
    unzip.extract(path)

In [23]:
ls 

fertilizer-rec.tgz  m1_hdf5.h5


In [24]:
!tar -zvcf fertilizer-rec.tgz m1_hdf5.h5

m1_hdf5.h5


In [25]:
ls -1

fertilizer-rec.tgz
m1_hdf5.h5


In [26]:
!pip install watson-machine-learning-client --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.9/538.9 kB 24.9 MB/s eta 0:00:00


In [27]:
from ibm_watson_machine_learning import APIClient
wml_cred = {
    "url" : "https://jp-tok.ml.cloud.ibm.com",
    "apikey" : "9grIdOQWMO4HNsgfzSCbQyeEShuRheeUFmKZ8nowYg5H"
}
client = APIClient(wml_cred)

In [28]:
def guid_from_space_name(client, space_name): 
    space = client.spaces.get_details() 
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [29]:
space_uid = guid_from_space_name(client, 'HIVE')
print("Space ID " + space_uid)

Space ID 5f9420b7-7e49-4a27-b868-c4e59872945d


In [30]:
client.set.default_space(space_uid)

'SUCCESS'

In [31]:
client.software_specifications.list(limit=500)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

In [32]:
software_spec_uid  = client.software_specifications.get_uid_by_name("tensorflow_rt22.2-py3.10") 
software_spec_uid

'f65bd165-f057-55de-b5cb-f97cf2c0f393'

In [33]:
model_details = client.repository.store_model(model='fertilizer-rec.tgz',meta_props={
    client.repository.ModelMetaNames.NAME: "HIVE",
    client.repository.ModelMetaNames.TYPE: "tensorflow_rt22.2",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid}
)

model_id = client.repository.get_model_id(model_details)

In [34]:
model_id

'cc8599f1-fe98-4d97-babf-c739231e27de'

In [35]:
client.repository.download(model_id, 'HIVE.tar.gz')

Successfully saved model content to file: 'HIVE.tar.gz'


'/home/wsuser/work/HIVE.tar.gz'